# Evaluating RL Algorithms for Autonomous Navigation in Webots

[Github Repository](https://github.com/grace-ortiz/COGS188_Final)

## Group members

- Katie Chung
- Jiawei Gao
- Grace Ortiz
- Hsiang-An Pao

# Abstract
This section should be short and clearly stated. It should be a single paragraph <200 words.  It should summarize:
- what your goal/problem is
- what the data used represents
- the solution/what you did
- major results you came up with (mention how results are measured)

__NB:__ this final project form is much more report-like than the proposal and the checkpoint. Think in terms of writing a paper with bits of code in the middle to make the plots/tables

# Background

Fill in the background and discuss the kind of prior work that has gone on in this research area here. **Use inline citation** to specify which references support which statements.  You can do that through HTML footnotes (demonstrated here). I used to reccommend Markdown footnotes (google is your friend) because they are simpler but recently I have had some problems with them working for me whereas HTML ones always work so far. So use the method that works for you, but do use inline citations.

Here is an example of inline citation. After government genocide in the 20th century, real birds were replaced with surveillance drones designed to look just like birds<a name="lorenz"></a>[<sup>[1]</sup>](#lorenznote). Use a minimum of 3 to 5 citations, but we prefer more <a name="admonish"></a>[<sup>[2]</sup>](#admonishnote). You need enough citations to fully explain and back up important facts.

Remeber you are trying to explain why someone would want to answer your question or why your hypothesis is in the form that you've stated.

# Problem Statement

Clearly describe the problem that you are solving. Avoid ambiguous words. The problem described should be well defined and should have at least one ML-relevant potential solution. Additionally, describe the problem thoroughly such that it is clear that the problem is quantifiable (the problem can be expressed in mathematical or logical terms), measurable (the problem can be measured by some metric and clearly observed), and replicable (the problem can be reproduced and occurs more than once).

# Data

The agent generated its own data through its intereaction with the Webots environment. Each observation from the agent consisted of various sensor readings:

| Sensor | Data Description | Raw Data Type | 
|---|---|---|
| Camera | BGRA image frame | 1D byte array |
| GPS | XYZ coordinates | 3D float array |
| LiDAR | SICK LMS 291 point cloud distance | 1D float array | 
| Gyro | 3-axis angular velocity| 3D float array |

The raw input data was processed into useable obserations for the state space. Due to the length of the data processing functions implemented, the process can be found in the in webots_env.py<a name="processing"></a>[<sup>[1]</sup>](#processingnote).

The BGRA image frames were processed by removing the alpha channel and converting the HSV color space [Figure 1], creating a lane mask to detect the yellow center line [Figure 2]. A region of interest focuses on the bottom half of the frame and Canny edge detection is used to determine the lane edges. Finally, a sliding window algorithm tracks lane positionsby identifying lane pixels in successive windows. Originally we attempted to use a combined mask to detect both the solid yellow lane line and the dotted white lane lines. However, due to the resolution of the simulated camera in Webots, the white line proved too difficult to detect consistently and was removed from the lane mask. The center of the lane was calculated by determining a fixed pixel distance from the yellow center lane line rather than the center of the two lane lines. 

<img src="final_imgs/frame.png" alt="WeBots Car Camera Frame" width="400">
<p align="left"><em>Figure 1: WeBots car camera frame</em></p>
<img src="final_imgs/lane_mask.png" alt="WeBots Car Camera Lane Mask" width="400">
<p align="left"><em>Figure 2: Assocated lane mask for WeBots frame</em></p>


The GPS x and y coordinates were extracted and passed to the state space, and the speed was extracted from the GPS and converted from m/s to km/h. The LiDAR data is processed by replacing any null values with a safe maxium distance of 100.0 km. The minimum obstacle distance is extracted and passed to the state space. Using the LiDAR's field of view (FOV) and the index of the minimum distance, the angle of the nearest obstacle is calculated and converted from radians to degrees. The gyroscope data is not used in the state space, however the x value is extracted and used to detect collisions by determining if the agent has flipped over. 

In [ ]:
# action space: [steering, speed]
self.action_space = spaces.Box(
    low=np.array([MIN_STEER_ANGLE, MIN_SPEED]), 
    high=np.array([MAX_STEER_ANGLE, MAX_SPEED]),
    dtype=np.float32
)

# state space 
self.state_space = spaces.Dict({
    "speed": spaces.Box(low=0, high=MAX_SPEED, shape=(1,), dtype=np.float32),  # gps speed
    "gps": spaces.Box(low=-np.inf, high=np.inf, shape=(2,), dtype=np.float32),  # (x, y) gps coordinates
    "lidar_dist": spaces.Box(low=0, high=100, shape=(1,), dtype=np.float32),  # distance to nearest obstacle
    "lidar_angle": spaces.Box(low=-90, high=90, shape=(1,), dtype=np.float32),  # angle to nearest obstacle
    "lane_deviation": spaces.Box(low=0, high=np.inf, shape=(1,), dtype=np.float32),  # pixels away from lane center
    "lane_mask": spaces.Box(low=0, high=1, shape=(64, 128, 1), dtype=np.uint8)  # binary mask for lane line (yellow line only)
})

The state space had 6 total variables: speed, gps, lidar_dist, lidar_angle, lane_deviation, lane_mask. The number of obervations was dependent on the number of episodes the agent trained and how many steps each epsiode lasted. Webots by default runs at 32ms per time step, meaning approximately 31 observations will be recorded per 1 second of simulation time. 


# Proposed Solution


To address the problem of training an autonomous self-driving vehicle in the Webots environment, we propose a solution based on four reinforcement learning algorithms: SARSA, Deep Q-Network (DQN), Proximal Policy Optimization (PPO), and Monte Carlo (MC) as a benchmark. Each algorithm is specifically adapted to handle the state observations provided by Webots (including speed, camera, lane deviation, and LIDAR readings) and is designed to optimize vehicle steering without accidents. The goal is to maximize cumulative rewards while ensuring adherence to traffic rules.




### Algorithmic Implementation Plan
1. SARSA (State-Action-Reward-State-Action)  -
SARSA will be used to learn an on-policy value function in a discretized state space. Continuous environment states will be discretized into bins. The agent selects actions using an epsilon-greedy strategy to balance exploration and exploitation. Q-values will be updated with Q(s,a) ← Q(s,a) + α [r + γQ(s’,a’) – Q(s,a)] ; This iterative update allows the agent to learn optimal policies in an on-policy manner. SARSA is reproducibly implemented using custom Python code integrated with Webots’ API , and results will be tested with various discretization granularities and learning rates.




2. DQN (Deep Q-Network) -
DQN will handle the continuous and high-dimensional state space using a neural network to approximate the Q-function. Neural network maps states to Q-values for each action. Experience Replay will store experiences in a replay buffer, sampled in mini-batches to stabilize training. Epsilon Decay helps educes exploration over time, shifting toward exploitation. The neural network and training loop will be implemented in PyTorch with reproducibility ensured via fixed random seeds and hyperparameter logging.  DQN is traditionally used for discrete action spaces, but it can be adapted for self-driving vehicle simulations, as some actions can be discretized.




3. PPO (Proximal Policy Optimization) -
PPO will directly optimize a parameterized policy for continuous control tasks, making it well-suited for vehicle navigation. This allows the agent to learn smooth and precise control policies, such as gradually adjusting the steering angle or smoothly accelerating/decelerating.[<sup>[7]</sup>](https://aryanjha.medium.com/creating-a-self-driving-car-simulation-977bed8f49b4) Policy Network allows Nneural networks trained to output action distributions and value estimates. THe clipped surrogate ensures stable policy updates by preventing excessively large updates. Generalized Advantage Estimation (GAE)reduces variance in policy gradient estimates with multiple epoch updates that increases sample efficiency.




4. MC (Monte Carlo) -
MC will be implemented as a benchmark to compare against the other RL methods. The algorithm will collect complete episode trajectories, compute returns at the end of each episode, and update Q-values based on the average return for each state-action pair. The Monte Carlo method will serve as the benchmark. While slower to converge, MC provides unbiased estimates of the value function and will be used to evaluate the relative performance of SARSA, DQN, and PPO in terms of convergence speed and cumulated rewards.




### Testing and Evaluation
All algorithms will be tested in the Webots simulation using the following metrics:




Episode Reward: The total accumulated reward per episode.
<br>
Convergence: Convergence across different RL algorithms would be compared to see how well each learned.
<br>
Success/Safety Metrics (TBD by training progress): The percentage of episodes where the vehicle reaches its goal without collisions. The number of collisions and lane deviation during stimlation.


# Evaluation Metrics

Propose at least one evaluation metric that can be used to quantify the performance of both the benchmark model and the solution model. The evaluation metric(s) you propose should be appropriate given the context of the data, the problem statement, and the intended solution. Describe how the evaluation metric(s) are derived and provide an example of their mathematical representations (if applicable). Complex evaluation metrics should be clearly defined and quantifiable (can be expressed in mathematical or logical terms).

# Results

You may have done tons of work on this. Not all of it belongs here.

Reports should have a __narrative__. Once you've looked through all your results over the quarter, decide on one main point and 2-4 secondary points you want us to understand. Include the detailed code and analysis results of those points only; you should spend more time/code/plots on your main point than the others.

If you went down any blind alleys that you later decided to not pursue, please don't abuse the TAs time by throwing in 81 lines of code and 4 plots related to something you actually abandoned.  Consider deleting things that are not important to your narrative.  If its slightly relevant to the narrative or you just want us to know you tried something, you could keep it in by summarizing the result in this report in a sentence or two, moving the actual analysis to another file in your repo, and providing us a link to that file.

### Sample Efficiency 
In our sample efficiency analysis, we compared Monte Carlo, SARSA, and Deep Q Network by measuring how quickly each alogorithm converged to stable behavior. Unfortunately, PPO failed to train properly due to issues with the environment preventing its inclusion in the comparison.  


### Differing Rewards Functions
have graph for sarsa and mc with diff reward functions


### Hyperparameter Exploration
A higher Alpha (0.1) and Gamma (0.99) indicate a stronger focus on faster convergence and greater emphasis on future rewards. In contrast, the second figure, with lower values, demonstrates slower but more stable convergence, placing more weight on immediate rewards. From Figure 1, we can observe higher rewards compared to Figure 2 as the number of episodes increases, along with a more stable growth trend. Meanwhile, the values in Figure 2 show minimal improvement over the course of training. Additionally, Figure 2 displays a smoother epsilon decay.

<img src="final_imgs/sarsa_a01.png" alt="WeBots Car Camera Frame" width="400">
<p align="left"><em>Figure 1: SARSA Higher Learning Rate& Discount</em></p>
<img src="final_imgs/sarsa_a05.png" alt="WeBots Car Camera Lane Mask" width="400">
<p align="left"><em>Figure 2: SARSA Lower Learning Rate& Discount</em></p>



# Discussion




### Interpreting the result




OK, you've given us quite a bit of tech informaiton above, now its time to tell us what to pay attention to in all that.  Think clearly about your results, decide on one main point and 2-4 secondary points you want us to understand. Highlight HOW your results support those points.  You probably want 2-5 sentences per point.








### Limitations




Are there any problems with the work?  For instance would more data change the nature of the problem? Would it be good to explore more hyperparams than you had time for? 








### Future work
Looking at the limitations and/or the toughest parts of the problem and/or the situations where the algorithm(s) did the worst... is there something you'd like to try to make these better.




### Ethics & Privacy




Ethical concerns in the development and deployment of self-driving vehicles are multifaceted and require careful consideration, with safety emerging as the foremost priority. Self-driving vehicles rely heavily on machine learning models to interpret their surroundings, make real-time decisions, and navigate complex environments. However, these models are not infallible and may encounter unpredictable or edge-case scenarios that could lead to accidents or unsafe outcomes. Unlike human drivers, AI systems lack personal accountability, raising significant questions about responsibility and liability in the event of failures. For instance, if an accident occurs due to a flaw in the ML model, determining who is at fault—whether it be the manufacturer, the software engineers, the vehicle owner, or even the regulatory bodies—becomes a complex legal and ethical challenge.[<sup>[8]</sup>](https://aryanjha.medium.com/creating-a-self-driving-car-simulation-977bed8f49b4).
This issue is further exacerbated if our model contributes to such failures, underscoring the need for robust testing, transparency, and accountability mechanisms.




One of the most profound ethical dilemmas in self-driving technology arises in unavoidable accident scenarios, often referred to as the "trolley problem."[<sup>[9]</sup>](https://montrealethics.ai/the-ethical-considerations-of-self-driving-cars/). In these situations, the AI system may be forced to choose between different harmful outcomes, such as prioritizing the safety of its passengers over pedestrians or other drivers. This raises difficult questions about how ethical guidelines should be programmed into the system. Should the vehicle prioritize the greater good, minimize harm, or protect its occupants at all costs?. These decisions are not only technically challenging to implement but also philosophically contentious, as they require engineers to encode moral principles into algorithms. This challenge highlights the broader difficulty of defining and standardizing ethical guidelines for AI systems, particularly in a way that aligns with societal values and legal frameworks.




Bias in machine learning models is another critical ethical consideration that cannot be overlooked. The performance of self-driving systems is heavily dependent on the quality and diversity of the training data. If the dataset used to train the ML models lacks representation of a wide range of pedestrian appearances, road conditions, weather scenarios, or cultural contexts, the AI may struggle to make fair and accurate decisions in real-world environments. For example, a model trained primarily on data from urban areas with specific demographics might underperform in rural settings or fail to recognize pedestrians with diverse physical characteristics.[<sup>[10]</sup>](https://bytes.scl.org/self-driving-cars-ethics-the-trolley-problem/)This could lead to unsafe or discriminatory outcomes, disproportionately affecting certain groups. To mitigate this risk, it is essential to ensure that training datasets are comprehensive, inclusive, and rigorously tested across diverse environments. Additionally, ongoing monitoring and updating of the models are necessary to address biases that may emerge over time.




Privacy is another significant ethical concern in the development of self-driving vehicles. These systems collect and process vast amounts of data, including real-time location information, passenger behavior, and even personal or vehicle-specific details. While this data is crucial for improving the performance and safety of AI systems, it also poses a risk to user privacy if not handled responsibly. Unauthorized access, data breaches, or misuse of this information could lead to serious privacy violations. To address this, we would ensure that all data collected is anonymized and securely stored, minimizing the risk of privacy leaks. Furthermore, clear policies and protocols must be established to govern data collection, usage, and sharing, ensuring compliance with privacy regulations and building trust with users.








### Conclusion




Reiterate your main point and in just a few sentences tell us how your results support it. Mention how this work would fit in the background/context of other work in this field if you can. Suggest directions for future work if you want to.


# Footnotes
<a name="processingnote"></a>1.[^](#processing): https://github.com/grace-ortiz/COGS188_Final/blob/main/webots/controllers/webots_env.py<br>
<a name="self_driving_sim_note"></a>7.^: https://aryanjha.medium.com/creating-a-self-driving-car-simulation-977bed8f49b4<br>
<a name="self_driving_ethics_note"></a>8.^: https://doi.org/10.1007/s13347-021-00464-5<br>
<a name="self_driving_ethics_article_note"></a>9.^: https://montrealethics.ai/the-ethical-considerations-of-self-driving-cars/<br>
<a name="trolley_problem_ethics_note"></a>10.^: https://bytes.scl.org/self-driving-cars-ethics-the-trolley-problem/<br>

